<a href="https://colab.research.google.com/github/aiguozhe01/DIC_Assignment/blob/master/Sprint_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint アンサンブル学習

## 【考察】

**アンサンブル学習とは・・・**\
複数の学習モデルを組み合わせる手法。
* 下階層：ベースモデル（base/weak learner/ genelizer）
* 上階層：メタモデル（meta learner/stacker）

-----------------------------------------------------------------------

* バギング：訓練データを分割して、複数の決定木モデルの推定値を評価する方法。
* ブレンディング：複数のモデルの推定値を使って評価する方法
* スタッキング：ブレンディングとバギングのハイブリッド手法。複数のモデルを用い、訓練データを分割して評価する方法。

(Bagging)
        * for 決定木
        * 過学習（訓練データに過剰適合しがち）
        * so 汎化誤差（generalization error）を減らしたい。
        * generalization error = variance（モデルの複雑さ）+ bias（真の関数とモデルのずれ）** 2 + noise
        * therefore Baggingが提案される。（単一の決定木だと高くなりがちなvarianceを下げる事ができる。
            1. 訓練データから離散一様分布に従い、ランダムな標本再抽出（bootstrap sampling）
            2. 分割した各サブセットに対して決定木を当てはめ、複数の決定木の結果を得る。
            3. 最後に多数決（回帰ならば平均）を行う。
            4. そもそも上記のアルゴリズムに特徴量サンプリングも追加して、RandomForestとなる。
            
        * for RandomForest
        * bootstrap法で作成した各々の決定木同士の相関 > 分岐で異なる特徴量を選ぶ決定木を生成するRandomForest
        * 結果、RandomForestだとvariance（分散）が下がり、Baggingより汎化性能（未知のテストデータに対する識別能力）が高くなる。

(Blending)
        * Bagging/RandomForestと違い、異なる予測モデルを組み合わせる。
            * 異なる予測モデル、特徴量、訓練データ、パラメータ（多数決、平均値、最大値、最小値 etc）
            
(Stacking)
        * 予測モデルの積み重ね。

疑問点
* 説明変数をsplitしたが、目的変数もsplitが必要ではないのか？


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# データ分割用
from sklearn.model_selection import train_test_split
# MSE算出用
from sklearn.metrics import mean_squared_error
# 線形回帰用
from sklearn import datasets, linear_model
# 決定木用
from sklearn.tree import DecisionTreeRegressor
# SVR用
from sklearn.svm import SVR
# ロジスティック回帰用
from sklearn.linear_model import LogisticRegression
# ランダムフォレスト用
from sklearn.ensemble import RandomForestRegressor
# 主成分分析器用
from sklearn.decomposition import PCA 
# 標準化用
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# 標準化と学習モデルを一元化するため
from sklearn.pipeline import make_pipeline

## データセットの用意

In [ ]:
# csvファイルを読み込む
data_set = pd.read_csv('..\Week4\house_prices_train.csv')

# 目的変数と説明変数とで区分する。
input_data = data_set[["GrLivArea", "YearBuilt"]]
target_data = data_set[["SalePrice"]]

# 説明変数を8:2で分割する。
input_train, input_test = train_test_split(input_data, test_size=0.2, random_state=0)

# 目的変数を8:2で分割する。
target_train, target_test = train_test_split(target_data, test_size=0.2, random_state=0)

########
# 標準化のインスタンス化
sscaler = preprocessing.StandardScaler()
sscaler_input = sscaler.fit(input_train)
input_train_ss = sscaler_input.transform(input_train)
input_test_ss = sscaler_input.transform(input_test)

sscaler_target = sscaler.fit(target_train)
target_train_ss = sscaler_target.transform(target_train)
target_test_ss = sscaler_target.transform(target_test)
########

# 主成分分析
pca = PCA()
pca_default = pca.fit(input_train)
pca_ss = pca.fit(input_train_ss)

input_train_pca = pca_default.transform(input_train)
input_train_ss_pca = pca_ss.transform(input_train_ss)

input_test_pca = pca_default.transform(input_test)
input_test_ss_pca = pca_ss.transform(input_test_ss)


# 【問題1】ブレンディングのスクラッチ実装

ブレンディングを3通りスクラッチ実装せよ。

* 比較対象として、単一モデルも用意する。
* 比較数値は精度の上下
    * 例）精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さいこと。
----------    
    
**比較3通り**

1. ロジスティック回帰 vs ロジスティック回帰（標準化）
2. ロジスティック回帰 vs ロジスティック回帰（標準化+主成分分析）
3. ロジスティック回帰 vs ロジスティック回帰 & 線形回帰 & 決定木 & SVR（線形カーネル）& ランダムフォレスト

## 線形回帰モデル

In [ ]:
# コントロール対照
# 線形回帰モデルをインスタンス化
linear_regr = linear_model.LinearRegression()
linear_regr_ss = linear_model.LinearRegression()
linear_regr_ss_pca = linear_model.LinearRegression()

# モデルを学習させる。
linear_regr.fit(input_train, target_train)
linear_regr_ss.fit(input_train_ss, target_train)
linear_regr_ss_pca.fit(input_train_ss_pca, target_train)

# 学習後のモデルを用い、推測を行う。
target_predict = linear_regr.predict(input_test)
target_predict_ss = linear_regr_ss.predict(input_test_ss)
target_predict_ss_pca = linear_regr_ss_pca.predict(input_test_ss_pca)


# MSEを算出する前に、推測結果の平均値を出す！！！
# The mean squared error
linear_regr_mse = mean_squared_error(target_test, target_predict)
linear_regr_ss_mse = mean_squared_error(target_test, target_predict_ss)
linear_regr_ss_pca_mse = mean_squared_error(target_test, target_predict_ss_pca)

print('線形回帰の平均二乗誤差（MSE）は: %.5e'% linear_regr_mse)
print('線形回帰の標準化済み平均二乗誤差（MSE）は：%.5e'% linear_regr_ss_mse)
print('線形回帰の標準化+PCA済み平均二乗誤差（MSE）は：%.5e'% linear_regr_ss_pca_mse)

# plot outputs
#
# plt.scatter(input_test.iloc[:,0], target_test, color='black')
# plt.plot(input_test.iloc[:,0].sort_values(), np.sort(target_predict, axis=0), color='blue', linewidth=3)
#
# plt.xticks(())
# plt.yticks(())
# plt.title('GrLivArea vs SalePrice')
#
# plt.show()

線形回帰の平均二乗誤差（MSE）は: 2.94207e+09
線形回帰の標準化済み平均二乗誤差（MSE）は：2.94207e+09
線形回帰の標準化+PCA済み平均二乗誤差（MSE）は：2.94207e+09


## 決定木モデル

In [ ]:
# 決定木（回帰）のMSEを算出

# 決定木モデルをインスタンス化
tree_regr = DecisionTreeRegressor(random_state=0)
tree_regr_ss = DecisionTreeRegressor(random_state=0)
tree_regr_ss_pca = DecisionTreeRegressor(random_state=0)

# モデルを学習させる。
tree_regr.fit(input_train, target_train)
tree_regr_ss.fit(input_train_ss, target_train)
tree_regr_ss_pca.fit(input_train_ss_pca, target_train)

# 学習後のモデルを用い、推測を行う。
target_predict = tree_regr.predict(input_test)
target_predict_ss = tree_regr_ss.predict(input_test_ss)
target_precict_ss_pca = tree_regr_ss_pca.predict(input_test_ss_pca)

# The mean squared error
tree_regr_mse = mean_squared_error(target_test, target_predict)
tree_regr_ss_mse = mean_squared_error(target_test, target_predict_ss)
tree_regr_ss_pca_mse = mean_squared_error(target_test, target_predict_ss_pca)

print(f'決定木（回帰）の平均二乗誤差（MSE）は:{tree_regr_mse: .5e}')
print(f'決定木（回帰）の標準化済み平均二乗誤差（MSE）は：%.5e'% tree_regr_ss_mse)
print(f'決定木（回帰）の標準化+PCA済み平均二乗誤差（MSE）は：%.5e'% tree_regr_ss_pca_mse)

# plt.scatter(input_test.iloc[:,0], target_test, color='black')
# plt.plot(input_test.iloc[:,0].sort_values(), np.sort(target_predict, axis=0), color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())
# plt.title('GrLivArea vs SalePrice')

# plt.show()

決定木（回帰）の平均二乗誤差（MSE）は: 3.00917e+09
決定木（回帰）の標準化済み平均二乗誤差（MSE）は：3.01586e+09
決定木（回帰）の標準化+PCA済み平均二乗誤差（MSE）は：2.94207e+09


## SVRモデル

In [ ]:
# SVRのMSEを算出

# SVRモデルをインスタンス化
sv_regr = SVR(C=1.0, kernel='linear', epsilon=0.1)
sv_regr_pca = SVR(C=1.0, kernel='linear', epsilon=0.1)
sv_regr_ss = SVR(C=1.0, kernel='linear', epsilon=0.1)
sv_regr_ss_pca = SVR(C=1.0, kernel='linear', epsilon=0.1)

# モデルを学習させる。
sv_regr.fit(input_train, target_train.iloc[:, 0]) # SVR的に学習する際の目的変数は1dが望ましいので、pandas.series化する。
sv_regr_pca.fit(input_train_pca, target_train.iloc[:, 0])
sv_regr_ss.fit(input_train_ss, target_train.iloc[:, 0])
sv_regr_ss_pca.fit(input_train_ss_pca, target_train.iloc[:, 0])


# 学習後のモデルを用い、推測を行う。
target_predict = sv_regr.predict(input_test)
target_predict_pca = sv_regr_pca.predict(input_test_pca)
target_predict_ss = sv_regr_ss.predict(input_test_ss)
target_predict_ss_pca = sv_regr_ss_pca.predict(input_test_ss_pca)

# The mean squared error
sv_regr_mse = mean_squared_error(target_test, target_predict)
sv_regr_pca_mse = mean_squared_error(target_test, target_predict)
sv_regr_ss_mse = mean_squared_error(target_test, target_predict_ss)
sv_regr_ss_pca_mse = mean_squared_error(target_test, target_predict_ss_pca)

print(f'SVRの平均二乗誤差（MSE）は:{sv_regr_mse: .3e}')
print(f'SVRのPCA済み平均二乗誤差（MSE）は：{sv_regr_pca_mse: .3e}')
print(f'SVRの標準化済み平均二乗誤差（MSE）は：%.3e'% sv_regr_ss_mse)
print(f'SVRの標準化+PCA済み平均二乗誤差（MSE）は：%.3e'% sv_regr_ss_pca_mse)


# plt.scatter(input_test.iloc[:,0], target_test, color='black')
# plt.plot(input_test.iloc[:,0].sort_values(), np.sort(target_predict, axis=0), color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())
# plt.title('GrLivArea vs SalePrice')

# plt.show()

SVRの平均二乗誤差（MSE）は: 2.942e+09
SVRのPCA済み平均二乗誤差（MSE）は： 2.942e+09
SVRの標準化済み平均二乗誤差（MSE）は：7.092e+09
SVRの標準化+PCA済み平均二乗誤差（MSE）は：7.092e+09


## ロジスティック回帰モデル

In [ ]:
# ロジスティック回帰のMSEを算出

# ロジスティック回帰モデルをインスタンス化
log_regr = LogisticRegression(random_state=0)
log_regr_ss = LogisticRegression(random_state=0)
log_regr_ss_pca = LogisticRegression(random_state=0)

# モデルを学習させる。
log_regr.fit(input_train, target_train.iloc[:, 0])
log_regr_ss.fit(input_train_ss, target_train.iloc[:, 0])
log_regr_ss_pca.fit(input_train_ss_pca, target_train.iloc[:, 0])

# 学習後のモデルを用い、推測を行う。
target_predict = log_regr.predict(input_test)
target_predict_ss = log_regr_ss.predict(input_test_ss)
target_predict_ss_pca = log_regr_ss.predict(input_test_ss_pca)

# The mean squared error
log_regr_mse = mean_squared_error(target_test, target_predict)
log_regr_ss_mse = mean_squared_error(target_test, target_predict_ss)
log_regr_ss_pca_mse = mean_squared_error(target_test, target_predict_ss_pca)

print(f'ロジスティック回帰の平均二乗誤差（MSE）は:{log_regr_mse: .3e}')
print(f'ロジスティック回帰の標準化済み平均二乗誤差（MSE）は：%.3e'% log_regr_ss_mse)
print(f'ロジスティック回帰の標準化+PCA済み平均二乗誤差（MSE）は：%.3e'% log_regr_ss_pca_mse)

ロジスティック回帰の平均二乗誤差（MSE）は: 5.471e+09
ロジスティック回帰の標準化済み平均二乗誤差（MSE）は：3.836e+09
ロジスティック回帰の標準化+PCA済み平均二乗誤差（MSE）は：4.878e+09


## ランダムフォレストモデル

In [ ]:
# ランダムフォレストのMSEを算出

# ランダムフォレストモデルをインスタンス化
forest_regr = RandomForestRegressor(max_depth=2, random_state=0)
forest_regr_ss = RandomForestRegressor(max_depth=2, random_state=0)
forest_regr_ss_pca = RandomForestRegressor(max_depth=2, random_state=0)

# モデルを学習させる。
forest_regr.fit(input_train, target_train.iloc[:, 0])
forest_regr_ss.fit(input_train_ss, target_train.iloc[:, 0])
forest_regr_ss_pca.fit(input_train_ss_pca, target_train.iloc[:, 0])

# 学習後のモデルを用い、推測を行う。
forest_target_predict = forest_regr.predict(input_test)
forest_target_predict_ss = forest_regr_ss.predict(input_test_ss)
forest_target_predict_ss_pca = forest_regr_ss_pca.predict(input_test_ss_pca)

# The mean squared error
forest_regr_mse = mean_squared_error(target_test, forest_target_predict)
forest_regr_ss_mse = mean_squared_error(target_test, forest_target_predict_ss)
forest_regr_ss_pca_mse = mean_squared_error(target_test, forest_target_predict_ss_pca)

print(f'ランダムフォレストの平均二乗誤差（MSE）は:{forest_regr_mse: .3e}')
print(f'ランダムフォレストの標準化済み平均二乗誤差（MSE）は：%.3e'% forest_regr_ss_mse)
print(f'ランダムフォレストの標準化+PCA済み平均二乗誤差（MSE）は%.3e'% forest_regr_ss_pca_mse)

ランダムフォレストの平均二乗誤差（MSE）は: 2.959e+09
ランダムフォレストの標準化済み平均二乗誤差（MSE）は：2.956e+09
ランダムフォレストの標準化+PCA済み平均二乗誤差（MSE）は2.474e+09


## 全モデルの作表

In [ ]:
df = pd.DataFrame(np.zeros(15).reshape(5, 3),
                 columns=['default', 'sscaler', 'sscaler+PCA'],
                 index=['linear_reg', 'tree', 'SVR', 'log_reg', 'forest'])
df.loc['linear_reg'] = [linear_regr_mse, linear_regr_ss_mse, linear_regr_ss_pca_mse]
df.loc['tree'] = [tree_regr_mse, tree_regr_ss_mse, tree_regr_ss_pca_mse]
df.loc['SVR'] = [sv_regr_mse, sv_regr_ss_mse, sv_regr_ss_pca_mse]
df.loc['log_reg'] = [log_regr_mse, log_regr_ss_mse, log_regr_ss_pca_mse]
df.loc['forest'] = [forest_regr_mse, forest_regr_ss_mse, forest_regr_ss_pca_mse]
df

,default,sscaler,sscaler+PCA
linear_reg,2.942067e+09,2.942067e+09,2.942067e+09
tree,3.009170e+09,3.015860e+09,2.942067e+09
SVR,2.941629e+09,7.092013e+09,7.092013e+09
log_reg,5.471145e+09,3.836195e+09,4.878052e+09
forest,2.958781e+09,2.956401e+09,2.473558e+09


## 結果

In [ ]:
# コントロール対象
print('ロジスティック回帰の平均二乗誤差（MSE）は: %.3e'% log_regr_mse)

# ロジスティック回帰前の標準化
print('比較対象①ロジスティック回帰（標準化）のMSEは：%.3e'% log_regr_ss_mse)

# ロジスティック回帰前に標準化と主成分分析
print('比較対象②ロジスティック回帰（標準化+主成分分析）の平均MSEは：%.3e'% log_regr_ss_pca_mse)

# 線形回帰+決定木+SVR+ロジスティック回帰+ランダムフォレストの平均MSE
set_3 = linear_regr_mse, tree_regr_mse, sv_regr_mse, log_regr_mse, forest_regr_mse
print('比較対象③ロジスティック回帰 & 線形回帰 & 決定木 & SVR（線形カーネル）& ランダムフォレストの平均MSEは：%.3e'% np.mean(set_3))
print()
print("結論：線形回帰のみのモデルがMSE値も低くかった。")

ロジスティック回帰の平均二乗誤差（MSE）は: 5.471e+09
比較対象①ロジスティック回帰（標準化）のMSEは：3.836e+09
比較対象②ロジスティック回帰（標準化+主成分分析）の平均MSEは：4.878e+09
比較対象③ロジスティック回帰 & 線形回帰 & 決定木 & SVR（線形カーネル）& ランダムフォレストの平均MSEは：3.465e+09

結論：線形回帰のみのモデルがMSE値も低くかった。


# 【問題2】バギングのスクラッチ実装

バギングをスクラッチ実装せよ。

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from numpy import random

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
house_data = pd.read_csv('/content/drive/My Drive/DIC/dataset/House_Prices/house_prices_train.csv')
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

train.head()

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [ ]:
# Blendingクラスの実装

class Blending():
    def __init__(self, model_list=None):
        self.model_list = model_list
    
    def fit(self, X, y):
        for model in self.model_list:
            model.fit(X, y)
            
    def predict(self, X, metric='mean', weight=None):
        
        preds = np.empty((len(X), len(self.model_list)))
        
        for i, model in enumerate(self.model_list):
            preds[:, i] = model.predict(X)
        
        
        if metric == 'mean':
            return preds.mean(axis=1)
        elif metric == 'median':
            return np.median(preds, axis=1)
        elif metric == 'max':
            return preds.max(axis=1)
        elif metric == 'min':
            return preds.min(axis=1)
        elif metric == 'weighted':
            return np.sum(preds*weight, axis=1) / np.sum(weight)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor()

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']

# 各モデルのブレンドモデルをインスタンス化
blend = Blending(models)

In [ ]:
# 特徴量を標準化
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# 目的変数を対数変換
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print('\n')

# ブレンドモデルの学習・推定
metrics = ['mean', 'median', 'max', 'min']

for metric in metrics:
    blend.fit(X_train_std, y_train_log)
    pred = blend.predict(X_test_std, metric=metric)
    print("Blended {} model MSE: {:.4f}".format(metric, mean_squared_error(y_test_log, pred)))

Linear Regression MSE: 0.0513
SVR MSE: 0.0454
Decision Tree MSE: 0.0865


Blended mean model MSE: 0.0449
Blended median model MSE: 0.0399
Blended max model MSE: 0.0661
Blended min model MSE: 0.0734


In [ ]:
from sklearn.model_selection import train_test_split

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])

# データの分割（今回は8：2の割合）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

(1168, 2)
(1168,)


In [ ]:
# 交差検証用関数を作成
def cross_validate(model, X, y, split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    
    for train_id, val_id in kf.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]

        # 学習、推定
        model.fit(train_X, train_y)
        pred = model.predict(val_X)
        
        results.append(mean_squared_error(val_y, pred))
    return np.array(results)

In [ ]:
# Baggingクラスの実装

import copy

class Bagger():
    def __init__(self, model, n_samples, n_bootstraps=10):
        self.model = model
        self.n_bootstraps = n_bootstraps
        self.n_samples = n_samples
        self.model_list = [copy.deepcopy(model) for _ in range(n_bootstraps)]  # Shallow Copyだと変数のみCopyされるため、Deepcopyでデータ毎複製
    

    def fit(self, X, y):
        
        for i in range(self.n_bootstraps):
            
            bag_id = random.randint(len(X), size=self.n_samples)
            
            self.model_list[i].fit(X[bag_id], y[bag_id])


    def predict(self, X, metric='mean'):
        
        preds = np.empty((len(X), len(self.model_list)))
        
        for i, model in enumerate(self.model_list):
            preds[:, i] = model.predict(X)
        
        if metric == 'mean':
            return preds.mean(axis=1)
        elif metric == 'median':
            return np.median(preds, axis=1)
        elif metric == 'max':
            return preds.max(axis=1)
        elif metric == 'min':
            return preds.min(axis=1)

In [ ]:
# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor()

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']

results_standalone = pd.DataFrame(index=[labels], columns=['train score', 'test score'])
results_bagging = pd.DataFrame(index=[labels], columns=['train score', 'test score'])


# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    
    # 5分割交差検証
    results_standalone['train score'][label] = cross_validate(model, X_train_std, y_train_log).mean()

    # テストデータ検証
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    results_standalone['test score'][label] = mean_squared_error(y_test_log, pred)
    
#     print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))
print('\n')


# Baggingモデルの学習・推定

for model, label in zip(models, labels):
        
    bagging = Bagger(model, n_samples=500, n_bootstraps=10)
    
    # 5分割交差検証
    results_bagging['train score'][label] = cross_validate(bagging, X_train_std, y_train_log).mean()

    bagging.fit(X_train_std, y_train_log)
    pred = bagging.predict(X_test_std)
    results_bagging['test score'][label] = mean_squared_error(y_test_log, pred)


#     print("Bagged {} Train MSE: {:.4f}".format(label, cross_validate(bagging, X_train_std, y_train_log).mean()))
#     print("Bagged {} Test MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print("単一モデルの交差検証＆推定結果")
print(results_standalone)
print('\n')
print("バギングモデルの交差検証＆推定結果")
print(results_bagging)



単一モデルの交差検証＆推定結果
                  train score test score
Linear Regression   0.0485434  0.0512843
SVR                  0.042024  0.0453771
Decision Tree       0.0764026  0.0874866


バギングモデルの交差検証＆推定結果
                  train score test score
Linear Regression    0.048783  0.0513998
SVR                 0.0419724  0.0499888
Decision Tree       0.0436182   0.048793


## 【問題3】スタッキングのスクラッチ実装

In [ ]:
from sklearn.model_selection import KFold

class Stacked():
    def __init__(self, base_models, final_model, k=3):
        self.base_models = base_models
        self.final_model = final_model
        self.k = k
        self.model_inst = [[copy.deepcopy(self.base_models[i]) for _ in range(k)] for i in range(len(self.base_models))]


    def fit(self, X, y):
        kfold = KFold(n_splits=self.k, shuffle=True)
        
        X_train_f = np.empty((len(X), len(self.base_models)))
        
        for i in range(len(self.model_inst)):
            for model, (train_id, test_id) in zip(self.model_inst[i], kfold.split(X)):
            
                X_train, y_train = X[train_id], y[train_id]
                X_test, y_test = X[test_id], y[test_id]

                model.fit(X_train, y_train)
                X_train_f[test_id, i] = model.predict(X_test)
        
        self.final_model.fit(X_train_f, y)
        
    
    def predict(self, X):
        X_test_f = np.empty((len(X), len(self.base_models)))
        
        for i in range(len(self.model_inst)):
            
            pred = np.empty((len(X), self.k))
            
            for j, model in enumerate(self.model_inst[i]):
                pred[:, j] = model.predict(X)
                
            X_test_f[:, i] = pred.mean(axis=1)
            
        return self.final_model.predict(X_test_f)

In [ ]:
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor(max_depth=10)

stacked = Stacked([tree, svr], linreg, k=10)

models = [linreg, svr, tree, stacked]
labels = ['Linear Regression', 'SVR', 'Decision Tree', 'Stacked']

results_stacked = pd.DataFrame(index=[labels], columns=['train score', 'test score'])

for model, label in zip(models, labels):
   # 5分割交差検証
    results_stacked['train score'][label] = cross_validate(model, X_train_std, y_train_log).mean()

    # テストデータ検証
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    results_stacked['test score'][label] = mean_squared_error(y_test_log, pred)
    
print("スタッキングモデルの交差検証＆推定結果")
print(results_stacked)

スタッキングモデルの交差検証＆推定結果
                  train score test score
Linear Regression   0.0485434  0.0512843
SVR                  0.042024  0.0453771
Decision Tree        0.064919  0.0750215
Stacked             0.0408428  0.0435241
